# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is solidly a classification problem, because we are trying to partition students into distinct categories, i.e. label each as either "needs intervention" or "does not need intervention". Regression is suitable when the output is a continuous value, like GPA (to stay on the topic of students), but that is not the case in this problem.

Which brings up a subtlety about this data set that has me concerned. Our data labels are not the same as our target labels. In the former case, students are either "passed" or "failed", and we're assuming there's a one-to-one correspondence between them and the model's output, i.e. "does not need intervention" and "needs intervention", respectively. Since the 'passed' column indicates if the student passed the final exam, then this data is almost certainly drawn ex post facto, after intervention was needed. But we're never going to use this model after students have already passed or failed! There is an implicit assumption that a sample of student data is not affected by the point in the school year in which it was taken. Which is largely true, but not necessarily for the continuous features.

This may appear as only a theoretical concern, but take an example where we put this model to work at the halfway mark in the school year (which is a good time to evaluate for potential failures, in my opinion). Let's also pretend that `absences` is an extremely influential dimension for the model. We could have a student Z who may be at risk of failing, with 10 absences so far. However, the data we trained on was drawn from a sample representing an entire school year, where, perhaps, at risk students had over 20 absences. When our model sees student Z, 10 absences may not be enough for the model to take notice.

In reality, this model is solving the following problem: "Given some student data from the end of the year, see if you can make an educated guess about which ones passed and which ones failed." Seeing as there's no way to obtain midyear data, I'll assume that solving the problem of guessing if a student passed is the same as detecting if that student needs intervention. But moving forward the district should collect student data at the point when the students would be evaluated, then label them after graduation.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [20]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [21]:
# Calculate number of students
n_students = student_data.shape[0]

# Calculate number of features
n_features = student_data.shape[1] - 1 # Subtract 1 to exclude target column

# Calculate passing students
n_passed = student_data[student_data.passed == 'yes'].shape[0]

# Calculate failing students
n_failed = student_data[student_data.passed == 'no'].shape[0]

# Calculate graduation rate
grad_rate = 100 * n_passed / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.00%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [22]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [23]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
        
            
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [24]:
# Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train
test_prop = 1.0 * num_test / (num_test + num_train) 

# Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, random_state=31, test_size=test_prop)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer:**
Support vector machines attempt to maximize the margin of a decision boundary while keeping all data points properly classified. In algorithmic terms, it's a constraint satisfaction problem, and Zheng et al. recently found success using an SVM to classify satellite images based on the irrigated crops within[1]. An SVM offers great flexibility in choice of kernel. With the right selection for the problem's domain, it can provide excellent results. And because there are usually only a few support vectors, computing the value of an input is often time efficient. The downside: it offers great flexibility in choice of kernel! With the wrong selection it will yield bad results, and it takes careful study to determine the best kernel. I like the idea of an SVM here because we are working in a high-dimensional space, and this is in the SVM wheelhouse[2].

AdaBoost is a funny name that stands for "Adaptive Boosting", but more importantly, it describes a meta-learner, meaning it creates a succession of "weak" learners, boosting the results by weighting the training set in between runs. It gives higher weight to the points classified incorrectly, thus forcing the model to focus on those points. In the wild, one might apply this to any classifier to increase performance. He and Huang et al. used AdaBoost in conjunction with an active shape model to perform 3D liver segmentation, helping to diagnose liver diseases[3]. AdaBoost's flexibility is an immediate advantage: just plug in your weak classifier, and a properly implemented AdaBoost can use it. Plus, it is "provably effective if it can find the weak [learners]"[4], that is, a learner that does better than chance in a given domain. Reflecting on the algorithm, there is a distinct weakness. Let's say your data has several outliers. AdaBoost will continue to make their weights heavier, and the latter of the weak learners will start overfitting to them. I like AdaBoost in this problem because it is considered the best default classifier (when paired with decision trees)[5].

I'll also be considering logistic regression - a classifier, despite the name - which is similar to linear regression in that we're creating a continuous function, but in this case, we're using it to output the probability of a label. Researchers in Ireland recently used this type of learner (the multinomial version) to analyze city noise, and which variables (day of week, location, etc.) are of most influence[6]. By utilizing different solvers (algorithms), this type of regression can yield good results in sparse and dense data sets, meaning you can apply it right away to a classification problem without much initial exploration. It's also been around since the late 1950s, leaving plenty of time for computer scientists to optimize algorithms for solving it. There was little material regarding the drawbacks to this type of learner, but some say it does poorly when there are many categorical variables[7]. That may give us pause in this problem, seeing as we have a number of categorical variables. Yet given my prior experience with this classifier, it trains and predicts fast, and this is desirable because we have a school with limited computational resources.

[1]: http://www.sciencedirect.com/science/article/pii/S0303243414001615
[2]: http://scikit-learn.org/stable/modules/svm.html
[3]: http://scitation.aip.org/content/aapm/journal/medphys/43/5/10.1118/1.4946817
[4]: https://www.cse.buffalo.edu/~jcorso/t/CSE555/files/lecture_boosting.pdf
[5]: https://arxiv.org/pdf/1312.6086v1.pdf
[6]: http://ascelibrary.org/doi/abs/10.1061/(ASCE)TE.1943-5436.0000843
[7]: http://www.edvancer.in/logistic-regression-vs-decision-trees-vs-svm-part2/

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [25]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [26]:
# Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

# Initialize the three models
model_tup = (SVC(random_state=55), AdaBoostClassifier(random_state=517), LogisticRegression(random_state=198))

# Set up the training set sizes
sample_100 = np.random.choice(300, 100, replace=False)
sample_200 = np.random.choice(300, 200, replace=False)

train_100 = (X_train.iloc[sample_100], y_train.iloc[sample_100])

train_200 = (X_train.iloc[sample_200], y_train.iloc[sample_200])

train_300 = (X_train, y_train)

training_sets = (train_100, train_200, train_300)

# Execute the 'train_predict' function for each classifier and each training set size
for model in model_tup:
    for aset in training_sets:
        train_predict(model, aset[0], aset[1], X_test, y_test)

Training a SVC using a training set size of 100. . .
Trained model in 0.0062 seconds
Made predictions in 0.0022 seconds.
F1 score for training set: 0.8667.
Made predictions in 0.0064 seconds.
F1 score for test set: 0.7808.
Training a SVC using a training set size of 200. . .
Trained model in 0.0159 seconds
Made predictions in 0.0080 seconds.
F1 score for training set: 0.8659.
Made predictions in 0.0039 seconds.
F1 score for test set: 0.7867.
Training a SVC using a training set size of 300. . .
Trained model in 0.0199 seconds
Made predictions in 0.0200 seconds.
F1 score for training set: 0.8644.
Made predictions in 0.0042 seconds.
F1 score for test set: 0.7755.
Training a AdaBoostClassifier using a training set size of 100. . .
Trained model in 0.2372 seconds
Made predictions in 0.0093 seconds.
F1 score for training set: 0.9848.
Made predictions in 0.0091 seconds.
F1 score for test set: 0.7541.
Training a AdaBoostClassifier using a training set size of 200. . .
Trained model in 0.2125 s

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Support Vector Classifier**  

| Training Set Size | Training Time (s) | Prediction Time (test) (s) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           .0019         |         .0012          |      0.8535      |     0.7552      |
| 200               |           .0045         |         .0022          |      0.8662      |     0.7838      |
| 300               |           .0114         |         .0024          |      0.8644      |     0.7755      |

** Classifer 2 - AdaBoost**  

| Training Set Size | Training Time (s) | Prediction Time (test) (s) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          .1236          |          .0038         |      0.9781      |     0.6829      |
| 200               |          .1153          |          .0104         |      0.8873      |     0.7121      |
| 300               |          .0903          |          .0040         |      0.8513      |     0.7970      |

** Classifer 3 - Logistic Regression**  

| Training Set Size | Training Time (s) | Prediction Time (test) (s) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          .0010          |         .0001          |      .8611       |      .7313      |
| 200               |          .0018          |         .0001          |      .8611       |      .7481      |
| 300               |          .0023          |         .0001          |      .8449       |      .7879      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** After analyzing various machine learning methods, I've determined that, given the data and our limited computing resources, a logistic regression, or LR, model will be best in determining which students will need intervention to help them graduate. Perhaps most important in this consideration is performance. How well can a logistic regressor actually predict? The regressor trained on 300 students achieves an F1 of 0.7879 without any tuning. For reference, F1 is a measure of accuracy between 0 and 1, with 1 being a perfect score. The support vector machine came close with a score of 0.7755, yet this score did not improve with more training points, where the logistic regressor did. This means that if we want to retrain it in the future using more student data, there is reason to believe the testing score may improve.

AdaBoost, yet another classifier, had a better F1 score than LR, at 0.7970 with 300 training points. Yet the time required to train AdaBoost is an order of magnitude larger. (Not that fitting and predicting on these models takes significant time. No model takes longer than a fifth of a second to train.) Of all the tested classifiers, LR was far and away the fastest in this regard, and was also the fastest at making predictions on new students. That means the district can expect good performance, even with limited computational resources, and can even refit the model with much more data, if necessary. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** I imagine most members of the board are familiar with the concept of the "line of best fit". It's a line that tries to describe the relationship between two variables, and indicates the extent to which those two variables - for example, year and life expectancy - move in tandem. This is what's known as linear regression. The resulting line is actually a function that answers a question: _What is the average global life expectancy for a given year?_ Continuing the example, we can give our function a year, let's say 2020, and it will give a prediction on the average life expectancy. Now mathematically, you can create a function that considers more than just year to make a prediction. Maybe you want to use GDP or population to also help predict life expectancy. That's OK: linear regression can handle.

Now that seems close to what we want. We want to create a function that considers many aspects of a student's life - number of absences, how often they study, access to internet at home - then make a prediction. There' one important difference: we're not looking for a number, we're looking for an answer to a yes/no question: _Does this student need help before graduating?_ Logistic regression is the answer. It too creates a function from a set of training data, but this function outputs a yes/no answer rather than a value from the number line, i.e. a discrete value as opposed to a continuous one. In short, logistic regression is like creating a "line of best fit" that classifies data points. The mathematics are somewhat different, but the metaphor is the same.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [33]:
# Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Create the parameters list you wish to tune
parameters = [
    {'solver': ['newton-cg'], 'C': [.01, .1, 1, 10, 100], 'fit_intercept': [True, False]},
    {'solver': ['lbfgs'], 'C': [.01, .1, 1, 10, 100], 'fit_intercept': [True, False]},
    {'solver': ['liblinear'], 'C': [.01, .1, 1, 10, 100], 'fit_intercept': [True, False], 'penalty': ['l1'], 'intercept_scaling': [.01,.1,1,10,100]},
    {'solver': ['liblinear'], 'C': [.01, .1, 1, 10, 100], 'fit_intercept': [True, False], 'penalty': ['l2'], 'intercept_scaling': [.01,.1,1,10,100], 'dual': [True, False]}
]

# Initialize the classifier
clf = LogisticRegression(random_state=55)

# Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer, verbose=1)

# Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print grid_obj.best_params_
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Fitting 3 folds for each of 170 candidates, totalling 510 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    2.6s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   12.9s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   17.8s


{'penalty': 'l1', 'C': 0.01, 'solver': 'liblinear', 'intercept_scaling': 0.01, 'fit_intercept': True}
Made predictions in 0.0007 seconds.
Tuned model has a training F1 score of 0.8119.
Made predictions in 0.0004 seconds.
Tuned model has a testing F1 score of 0.7742.


[Parallel(n_jobs=1)]: Done 510 out of 510 | elapsed:   18.9s finished


### Question 5 - Final F<sub>1</sub> Score 
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The tuned model has about the same F1 socres for both the training and testing sets (for a model trained on 300 points). They are strictly lower, but this discrepancy is likely noise in the random state of the classifier. Of note is that the best tuned model uses a liblinear solver, which the scikit documentation indicates is best when dealing with small training sets. This model is going to need significant tuning if the board wants to reliably reach its goal of 95% graduation, else it should introduce some human verification, a sanity check of sorts. If a teacher sees a student that the model says does not need intervention, but some factor beyond our feature set seems troubling, that teacher should speak out.

In a future iteration, we should be supremely interested in increasing our recall, i.e. reducing false negatives, as much as possible. It's upsetting if we fail to intervene when a student needs help, but not too terrible if we aid a student that was doing just fine on her own.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to **File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.